In [ ]:
#%% 
#helper function to generate dropdown
def generate_dropdown(dropdown_type,dropdown_list):
    return dcc.Dropdown(
        id = dropdown_type,
        options=[{'label': i, 'value': i} for i in dropdown_list],
        value="",
        placeholder="Select a "+dropdown_type,
        style={'width':'100%','text-align': 'center'},
    )

# helper function for updating dropdown options
def update_dropdown(str_out,dict_in,year):
    # valid dataframe for the given year, excludes unranked companies of that year 
    df_year = df_new.loc[~pd.isna(df_new['Rank ' + str(year)])]
    # update valid dataframe based on dropdown inputs
    for key in dict_in.keys():
        if dict_in[key]: # only update if dropdown input
            df_year = df_year.loc[df_year[key]==dict_in[key]]
                
    # update options for output specified in str_out
    opt_list = sorted(list(set(df_year[str_out])))
    opts = [{'label': i, 'value': i} for i in opt_list]
    return opts

#helper function to get closest companies
def get_closest_companies(company,df):
    closest_companies = [company]
    for i in range(1,5):
        cc = (df_new[df_new['Company']==company]['Closest Company ' +str(i)]).tolist()[0]
        closest_companies.append(cc)
    return closest_companies

# dash app
import pandas as pd
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.graph_objects as go
import plotly.express as px

df_new = pd.read_pickle('pickles/dataframe_for_dash')
years = range(2008,2022)
metrics = ['Sales','Assets','Profits','Market Value'] #metrics for radio buttons
dropdowns = ['Country','Sector','Company'] #options for dropdowns
tabs = dropdowns.copy().append('Cluster') #tabs

# list of all options for dropdowns
sectors = sorted(list(set(df_new['Sector'])))
companies = sorted(list(df_new['Company']))
countries = sorted(list(df_new['Country']))

# font in the application
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash()

app.layout = html.Div([  # entire layout
    html.Div([    # Title + world-map + year-slider
        html.H2(['Business Analytics Dashboard (Forbes Global 2000)'],
                style={'text-align':'center','width':'100%','margin-bottom':'0%'}),
        dcc.Graph(
            id='world-map',
            #clickData={'points': [{'location': ''}]},
            style = {'width':'100%'}
        ),
        dcc.Slider(       
            id='year-slider',
            min=years[0],
            max=years[-1],
            value=years[-3],
            # 2008-2021
            marks={str(year): str(year) for year in years},
            step=None,
        )
    ],style={'margin-left':'5%','width': '80%','display':'block'}),  # World-map + year-slider
                                          
    html.Br(), html.Br(), html.Br(),
    
    #Radio buttons
    html.Div([      # radiobutton: 'metric' 
        dcc.RadioItems(id='metric',
            options = [{'label': i, 'value': i} for i in metrics],
            value = metrics[0],
            style={'font-size':'large','width':'80%','margin-left':'5%'}
        )
    ],style={'text-align':'center'}),           # radiobutton: 'metric'
    
    html.Br(), html.Br(), html.Br(),
    
    #Drop downs 
    html.Div([    # dropdown: 'Country'    
        generate_dropdown('country',countries)
    ],style={'display':'inline-block','width':'25%'}),
    
    html.Div([    # dropdown: 'sector'      
        generate_dropdown('sector',sectors)     
    ],style={'display':'inline-block','width':'25%'}),
    
    html.Div([    # dropdown: 'Company'    
        generate_dropdown('company',companies)
    ],style={'display':'inline-block','width':'25%'}),  
    
    html.Br(), html.Br(),
    
    html.Div([    # tab switches
        # 4 tabs for 4 different displays 
        dcc.Tabs(id='tabs', value='tab-country', children=[
            dcc.Tab(label='Country',value='tab-country'),
            dcc.Tab(label='Sector',value='tab-sector'),
            dcc.Tab(label='Company',value='tab-company'),
            dcc.Tab(label='Cluster',value='tab-cluster')
            ]),
        html.Div(id='tabs-content')
    ]),           # tab switches
        
])            # entire layout



# generate world map
# input: year-slider, metic
# output: world map
@app.callback(
    Output('world-map','figure'),
    Input('year-slider', 'value'),
    Input('metric','value')
)

def generate_worldmap(year,metric):
    if not metric:
        metric = metrics[0]
    year_metric = metric+ ' '+ str(year)
    df_year = df_new.loc[~pd.isna(df_new['Rank ' + str(year)])]
    df_country_sum = df_year.groupby('ISO Code').sum().reset_index()
    fig = go.Figure(go.Choropleth(
        locations=df_country_sum['ISO Code'], # Spatial coordinates
        z = df_country_sum[year_metric],
        colorscale = 'Reds',
        colorbar_title = "Billions USD",
        visible = True,
    ))
    fig.update_layout(title_text = year_metric)
    return fig

# update sector dropdown section 
# input: country, company, year-slider
# output: sector
@app.callback(
    Output('sector', 'options'),
    Input('year-slider', 'value'),
    Input('country', 'value'),
    Input('company', 'value')
    )

def update_dropdown_sector(year,country,company):
    dict_in = {'Country':country,'Company':company}
    str_out = 'Sector'
    return update_dropdown(str_out,dict_in,year)
    

# update country dropdown section
# input: sector, company, year-slider
# output: country
@app.callback(
    Output('country', 'options'),
    Input('year-slider', 'value'),
    Input('sector', 'value'),
    Input('company', 'value')
    )

def update_dropdown_country(year,sector,company):
    dict_in = {'Sector':sector,'Company':company}
    str_out = 'Country'
    return update_dropdown(str_out,dict_in,year)
    
    
# update company dropdown section
# input: sector, country, year-slider
# output: company
@app.callback(
    Output('company', 'options'),
    Input('year-slider', 'value'),
    Input('sector', 'value'),
    Input('country', 'value')
    )

def update_dropdown_company(year,sector,country):
    dict_in = {'Sector':sector,'Country':country}
    str_out = 'Company'
    return update_dropdown(str_out,dict_in,year)


# render tab content
# input: year-slider, sector, country, company, metric, tabs
# output: content of selected tab
@app.callback(
    Output('tabs-content', 'children'),
    Input('year-slider', 'value'),
    Input('sector', 'value'),
    Input('country', 'value'), 
    Input('company', 'value'),
    Input('metric', 'value'),
    Input('tabs', 'value')
    )

def render_content(year,sector,country,company,metric,tab):
    df_year = df_new.loc[~pd.isna(df_new['Rank ' + str(year)])]
    if not metric:
        metric = metrics[0]
    
    if tab == 'tab-country' or tab == 'tab-sector': 
        fig1,error = generate_graph(year,sector,country,metric,tab)
        if error:
            return html.Div([fig1])
        else:
            fig2 = generate_bargraph_year(year,sector,country,metric,company,tab)
            return html.Div([
                 dcc.Graph(figure=fig1),
                 dcc.Graph(figure=fig2)
            ])        
    elif tab == 'tab-company' or tab == 'tab-cluster': #company based graphs
        if company:
            if (df_year['Company'] != company).all():    
                return html.Div([html.H3('Please select a valid company for this year!')])
            elif tab == 'tab-company':
                return html.Div([
                     dcc.Graph(figure=generate_company_graph(year,company,tab)),
                     dcc.Graph(figure=generate_bargraph_year(year,sector,country,metric,company,tab))
                ])
            elif tab == 'tab-cluster':
                return html.Div([
                    dcc.Graph(figure=generate_company_graph(year,company,tab))
                ])


    
#helper function to restrict options when dropdown has input
def check_dropdown(country,sector,df_year):
    # if country dropdown has input
    if country:
        country_title = country
        df_country = df_year.loc[df_year['Country']==country]
    else:
        country_title = 'World'
        df_country = df_year
        
    # if sector dropdown has input
    if sector:
        sector_title = sector
        df_sector = df_country.loc[df_country['Sector']==sector]
    else:
        sector_title = 'All Sectors'
        df_sector = df_country
        
    return df_sector, country_title, sector_title
    

# helper function that generates graphs in 'tab-country' and 'tab-sector'
# input: year, country, sector metric
# output: fig
def checkForEmpty(df_sum,message):
    if df_sum.empty:
        output = html.H3('Please select a valid '+message+' for this year')
        return output

def generate_graph(year,sector,country,metric,tab):
    df_year = df_new.loc[~pd.isna(df_new['Rank ' + str(year)])]
    year_metric = metric + ' ' + str(year)
    error = True
    if tab == 'tab-country': #generate pie chart grouped by sector
        df_out, country_title, sector_title = check_dropdown(country,'',df_year)
        df_sum = df_out.groupby('Sector').sum().reset_index()
        fig = checkForEmpty(df_sum,'country')
        if fig is None:
            fig = px.pie(df_sum, values=year_metric, names='Sector')
            fig.update_layout(title_text = year_metric+' '+ country_title)
            error = False
    
    elif tab == 'tab-sector': #generate bar chart grouped by industry 
        df_out, country_title, sector_title = check_dropdown(country,sector,df_year)
        df_sum = df_out.groupby('Industry').sum().reset_index()
        fig = checkForEmpty(df_sum,'country and sector')
        if fig is None:
            fig = px.bar(df_sum,y=year_metric,x='Industry',color='Industry')
            fig.update_yaxes(title_text = year_metric+" (Billions USD)")
            fig.update_layout(title_text = country_title + ' '+ sector_title)  
            error = False
    return fig, error
    

# helper function that generates bargraph and clusters for companies 'tab-company'
# input: year, company
# output: fig(bar-graph)
def generate_company_graph(year,company,tab): 
    
    #get closest companies
    closest_companies = get_closest_companies(company,df_new)
    df_companies = df_new.loc[df_new['Company'].isin(closest_companies)]
    if tab == 'tab-cluster': #3D cluster plot
        fig = px.scatter_3d(df_companies, x="PCA_1 "+str(year), y="PCA_2 "+str(year), z="PCA_3 "+str(year),
                    color= "pca_cluster "+str(year), hover_data=["PCA_1 "+str(year), "PCA_2 "+str(year),"PCA_3 "+str(year), "Company"])
    elif tab == 'tab-company': #2D bar plot
        year_metrics = [metric + ' ' + str(year) for metric in metrics]    
        temp = df_companies[year_metrics].T
        temp.columns = df_companies["Company"]
    
        fig = go.Figure()
        for [i,col] in enumerate(temp.columns):
            fig.add_trace(go.Bar(x=temp.index,y=temp[col],name=col,showlegend=True))
        
        fig.update_xaxes(
            ticktext = [metric for metric in metrics],
            tickvals = [i for i in range(len(metrics))],
        )
    fig.update_layout(title_text = company +': Closest Companies')
    return fig

# helper function that generates lower bargraph(trend across years)
# in 'tab-country','tab-sector','tab-company'
# input: year,sector,country,metric,company,tab
# output: fig(bar-graph)
def generate_bargraph_year(year,sector,country,metric,company,tab):
    
    # dataframe for given (year,country,metric)
    df_year = df_new.loc[~pd.isna(df_new['Rank ' + str(year)])]
    
    # if country dropdown has input
    if country:
        df_country = df_year.loc[df_year['Country']==country]
        country_title = country
    else:
        country_title = 'World'
        df_country = df_year
            
    # compute sector sum of given metric
    year_metrics = [metric + ' ' + str(year) for year in years]
    
    if tab == 'tab-country':
        df_sum = df_country.groupby('Sector').sum().reset_index().sort_values(by=metric+' '+str(year),ascending=False)
        temp = df_sum[year_metrics].T
        temp.columns = df_sum["Sector"]
    
    elif tab == 'tab-sector':
        if sector:
            df_country = df_country.loc[df_country['Sector']==sector]
        df_sum = df_country.groupby('Industry').sum().reset_index()
        temp = df_sum[year_metrics].T
        temp.columns = df_sum['Industry']
    
    elif tab == 'tab-company':
        closest_companies = get_closest_companies(company,df_new)
        df_company = df_year.loc[df_year['Company'].isin(closest_companies)]
        temp = df_company[year_metrics].T
        temp.columns = df_company["Company"]
        
    fig = go.Figure()
    for [i,col] in enumerate(temp.columns):
        fig.add_trace(go.Bar(x=temp.index,y=temp[col],name=col,showlegend=True))
            
    fig.update_xaxes(
        ticktext=[str(year) for year in years],
        tickvals = [i for i in range(len(years))],
    )
    fig.update_yaxes(title_text=metric+" (Billions USD)")

    return fig
   
app.run_server(debug=True, use_reloader=False) 

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
